In [27]:
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import seaborn as sns
sns.set()

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os.path
import scipy as sp
import scipy.stats
import statsmodels as smr
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
scipy.stats.chisqprob = lambda chisq, df: scipy.stats.chi2.sf(chisq, df)

In [207]:
class Project:   
    def __init__(self):
        self.data = None
        self.train_data = None
        self.test_data = None
            
    def _load_data_(self,filepath):
      
        #if (~os.path.isfile('train.csv')) | (~os.path.isfile('test.csv')):
           # self.data = pd.read_csv(filepath,encoding='ISO-8859-1')
           # self.train_test_split()
           # self._write_data_()
            
        self.train_data = pd.read_csv('train.csv')
                                #,encoding='ISO-8859-1')
        self.test_data = pd.read_csv('test.csv',encoding='ISO-8859-1')
        return self.train_data
        
    def _write_data_(self):
        self.train_data.to_csv('train.csv')
        self.test_data.to_csv('test.csv')
    
    def train_test_split(self):
        self.train_data, self.test_data = train_test_split(self.data, test_size=0.2, random_state=42)   
        
    def null_summary(self,df):
        print(df.isnull().sum())
        all_data_na = (df.isnull().sum() / len(df)) * 100
        all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
        missing_data.head()
        if missing_data.size> 0:
            f, ax = plt.subplots(figsize=(15, 12))
            plt.xticks(rotation='90')
            sns.barplot(x=all_data_na.index, y=all_data_na)
            plt.xlabel('Features', fontsize=15)
            plt.ylabel('Percent of missing values', fontsize=15)
            plt.title('Percent missing data by feature', fontsize=15)
            plt.show()
            
    def get_stats(self,df,variable):
        print("Skewness: %f" % df[variable].skew())
        print("Kurtosis: %f" % df[variable].kurt())
        print("Mean: %f" % df[variable].mean())
        print("Variance: %f" % (df[variable].var()))

            
    def correlation_map(self,df,outcome,k):
        corrmat = df.corr()
        col_large = corrmat.nlargest(k+1,outcome)[outcome].index
        col_small = corrmat.nsmallest(k,outcome)[outcome].index
        cols = col_large.union(col_small)
        cm = np.corrcoef(corrmat[cols].loc[cols].values.T)
        print(corrmat[cols].loc[outcome])
        #sns.set(font_scale=1.25)
        hm = sns.heatmap(cm, cbar=True, vmax=1,annot=True, fmt='.1f', annot_kws={'size': 8}, yticklabels=cols.values, xticklabels=cols.values)
        return cols
    
    def encode_organization(self,x):
            if 'Academy' in x:
                return 1
            elif 'Instituion' in x:
                return 2
            elif 'College' in x:
                return 3
            elif 'University' in x:
                return 4
            else:
                return 0
    
    def encode_cost(self,df):
        prices = []
        for i in range(len(df)):
            if pd.notnull(df['NPT4_PUB'][i]):
                prices.append(df['NPT4_PUB'][i])
            elif pd.notnull(df['NPT4_PRIV'][i]):
                prices.append(df['NPT4_PRIV'][i])
            else:
                prices.append(np.NaN) 
        
        df.loc[:,'NET_COST'] = pd.Series(prices, index=df.index)
        
        return df
    
    def encode_25KBinary(self, df):
        vals = []
        for i in range(len(df)):
            if float(df['gt_25k_p6'][i]) >= 0.6:
                vals.append(1)
            elif float(df['gt_25k_p6'][i]) < 0.6:
                vals.append(0)
            else:
                vals.append(np.NaN)
        
        df.loc[:,'Binary_25k'] = pd.Series(vals, index=df.index)
        
        return df
    
    def encode_state(self,x):
        west = ['WA','MT','OR','ID','WT','CA','NV','UT','CO','AZ','NM']
        midwest = ['ND','MN','SD','NE','KS','IA','MO','WI','IL','MI','IN','OH','WY']
        northeast = ['NY','PA','NJ','RI','CT','MA','VT','NH','ME']
        south = ['TX','OK','AR','LA','MS','AL','GA','TN','KY','WV','DC','MD','VA','DE','NC','SC','FL']
        others =['AK','HI','PR','PW','AS','GU','FM','VI']
        
        if x in others:
            return 0
        elif x in midwest:
            return 1
        elif x in south:
            return 2
        elif x in west:
            return 3
        elif x in northeast:
            return 4
        else: return x
        

In [208]:
project = Project()
project._load_data_('train.csv')

,Unnamed: 0,INSTNM,CITY,STABBR,PREDDEG,CONTROL,LOCALE,SATVRMID,SATMTMID,SATWRMID,...,RET_PTL4,PCTFLOAN,UG25abv,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_4_POOLED_SUPP,C200_L4_POOLED_SUPP,md_earn_wne_p10,gt_25k_p6
0,7440,Marian Health Careers Center-Van Nuys Campus,Van Nuys,CA,1,3,11.0,NaN,NaN,NaN,...,NaN,0.6786,0.5745,15680,174.080144,0.847161572,NaN,0.27471978,47500,0.767567575
1,5144,Emma's Beauty Academy-Juana Diaz,Juana Diaz,PR,1,3,23.0,NaN,NaN,NaN,...,NaN,0.0000,0.1733,NaN,NaN,PrivacySuppressed,NaN,0.703098375,13400,0.181818187
2,7169,Milan Institute-Nampa,Nampa,ID,1,3,22.0,NaN,NaN,NaN,...,NaN,0.4332,0.4314,9500,105.469475,0.351691581,NaN,NaN,16100,0.249343827
3,5160,University of Phoenix-Bay Area Campus,San Jose,CA,3,3,11.0,NaN,NaN,NaN,...,NaN,0.6299,0.8464,35500,394.122775,0.406534043,0.135344954,NaN,53400,0.705231071
4,4987,Sanford-Brown Institute-Tampa,Tampa,FL,1,3,21.0,NaN,NaN,NaN,...,NaN,0.6094,0.4858,9500,105.469475,0.409117821,NaN,0.596397059,28300,0.524061024
5,2415,Truckee Meadows Community College,Reno,NV,2,1,12.0,NaN,NaN,NaN,...,0.4680,0.1438,0.3980,10000,111.0205,0.551597052,NaN,0.203587983,32500,0.543252587
6,6076,Healing Arts Institute,Perrysburg,OH,1,3,21.0,NaN,NaN,NaN,...,NaN,0.6197,0.6250,7917,87.89492985,0.755102041,NaN,NaN,NaN,PrivacySuppressed
7,1790,Maryland University of Integrative Health,Laurel,MD,4,2,21.0,NaN,NaN,NaN,...,NaN,NaN,NaN,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,NaN,NaN,NaN
8,7170,Milan Institute of Cosmetology-El Paso,El Paso,TX,1,3,11.0,NaN,NaN,NaN,...,NaN,0.4466,0.4847,14252,158.2264166,0.291276491,NaN,NaN,18900,0.228511527
9,3857,Middle Tennessee State University,Murfreesboro,TN,3,1,12.0,515.0,510.0,NaN,...,NaN,0.5732,0.2315,22170.5,246.1379995,0.739414441,0.453413673,NaN,36100,0.592180908


In [209]:
df = project.train_data
df = df.drop(df.columns[0],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6243 entries, 0 to 6242
Data columns (total 42 columns):
INSTNM                    6243 non-null object
CITY                      6243 non-null object
STABBR                    6243 non-null object
PREDDEG                   6243 non-null int64
CONTROL                   6243 non-null int64
LOCALE                    5911 non-null float64
SATVRMID                  1036 non-null float64
SATMTMID                  1048 non-null float64
SATWRMID                  631 non-null float64
ACTCMMID                  1076 non-null float64
ACTENMID                  939 non-null float64
ACTMTMID                  940 non-null float64
ACTWRMID                  247 non-null float64
SAT_AVG                   1136 non-null float64
DISTANCEONLY              5913 non-null float64
UGDS                      5680 non-null float64
UGDS_WHITE                5680 non-null float64
UGDS_BLACK                5680 non-null float64
UGDS_HISP                 5680 non-null 

In [211]:
df.shape

(6243, 42)

In [212]:
df = df.apply(lambda x: x.replace('PrivacySuppressed',np.NAN))
df = project.encode_cost(df)
df = project.encode_25KBinary(df)

In [213]:
df['RPY_3YR_RT_SUPP'] = pd.to_numeric(df['RPY_3YR_RT_SUPP'])
df['md_earn_wne_p10'] = pd.to_numeric(df['md_earn_wne_p10'])
df['GRAD_DEBT_MDN_SUPP'] = pd.to_numeric(df['GRAD_DEBT_MDN_SUPP'])
df['GRAD_DEBT_MDN10YR_SUPP'] = pd.to_numeric(df['GRAD_DEBT_MDN10YR_SUPP'])
df['C150_4_POOLED_SUPP'] = pd.to_numeric(df['C150_4_POOLED_SUPP'])
df['C200_L4_POOLED_SUPP'] = pd.to_numeric(df['C200_L4_POOLED_SUPP'])

In [214]:
columns = df.columns
#drop outcomes variables and "name" variables 
treatment_variables = columns.drop(['INSTNM','CITY','STABBR','RPY_3YR_RT_SUPP', 'md_earn_wne_p10', 'gt_25k_p6', 'Binary_25k'])

In [215]:
#only keep the variables if less than 20% of the values are missing ( SHOULD I KEEP IT at 20%?)
keep_variables = []
for i in range(len(treatment_variables)):
    x = df[str(treatment_variables[i])].isnull().sum()/float((len(df)))
    if x <= 0.2:
        keep_variables.append(treatment_variables[i]) 

In [216]:
keep_variables

['PREDDEG',
 'CONTROL',
 'LOCALE',
 'DISTANCEONLY',
 'UGDS',
 'UGDS_WHITE',
 'UGDS_BLACK',
 'UGDS_HISP',
 'UGDS_ASIAN',
 'UGDS_AIAN',
 'UGDS_NHPI',
 'UGDS_2MOR',
 'UGDS_NRA',
 'UGDS_UNKN',
 'PPTUG_EF',
 'PCTPELL',
 'PCTFLOAN',
 'UG25abv',
 'GRAD_DEBT_MDN_SUPP',
 'GRAD_DEBT_MDN10YR_SUPP',
 'NET_COST']

In [50]:
#correlations for RPY_3YR_RT_SUPP
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['RPY_3YR_RT_SUPP'].corr(df[str(keep_variables[i])])

PREDDEG
0.41195206057269423
CONTROL
-0.3261100801479304
LOCALE
0.08168255497972415
DISTANCEONLY
0.05069499664203906
UGDS
0.1760012325037092
UGDS_WHITE
0.46492732204439524
UGDS_BLACK
-0.4054922206913137
UGDS_HISP
-0.24925158539030579
UGDS_ASIAN
0.21773867892349655
UGDS_AIAN
-0.06694708969820343
UGDS_NHPI
-0.05706172562457254
UGDS_2MOR
0.03307421457283581
UGDS_NRA
0.2809839411391704
UGDS_UNKN
-0.06485775203603199
PPTUG_EF
-0.06273166447098115
PCTPELL
-0.5249647263345588
PCTFLOAN
-0.03015274997764476
UG25abv
-0.46985207003761786
GRAD_DEBT_MDN_SUPP
0.17582157858338796
GRAD_DEBT_MDN10YR_SUPP
0.17582157855883793
NET_COST
0.16511478937228596


In [51]:
#regressions for RPY_3YR_RT_SUPP
for i in range(len(keep_variables)):
    print (i)
    print (smf.ols('RPY_3YR_RT_SUPP ~ ' + str(keep_variables[i]), data=df).fit().summary())

0
                            OLS Regression Results                            
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     54.78
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           1.75e-12
Time:                        22:32:21   Log-Likelihood:                 88.151
No. Observations:                 270   AIC:                            -172.3
Df Residuals:                     268   BIC:                            -165.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4430      0.022     20.345      0

                            OLS Regression Results                            
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     21.60
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           5.42e-06
Time:                        22:32:22   Log-Likelihood:                 69.268
No. Observations:                 254   AIC:                            -134.5
Df Residuals:                     252   BIC:                            -127.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5705      0.012     46.373      0.0

In [52]:
#correlations for md_earn_wne_p10
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['md_earn_wne_p10'].corr(df[str(keep_variables[i])])

PREDDEG
0.4655136713650063
CONTROL
-0.15736524407428276
LOCALE
-0.041946584251139785
DISTANCEONLY
0.009427700774562528
UGDS
0.2513840066266706
UGDS_WHITE
0.19380623900842853
UGDS_BLACK
-0.1408290566352015
UGDS_HISP
-0.1937505670598538
UGDS_ASIAN
0.3324676445649233
UGDS_AIAN
-0.10540507785989933
UGDS_NHPI
0.03930500193618835
UGDS_2MOR
-0.0193596184968103
UGDS_NRA
0.31990139837371884
UGDS_UNKN
0.04523559217341784
PPTUG_EF
-0.006521897026611991
PCTPELL
-0.3820015587477176
PCTFLOAN
0.1808783740872359
UG25abv
-0.18441329942042267
GRAD_DEBT_MDN_SUPP
0.5688044013741512
GRAD_DEBT_MDN10YR_SUPP
0.5688044013712731
NET_COST
0.33818051379041075


In [53]:
#regressions for md_earn_wne_p10
for i in range(len(keep_variables)):
    print (i)
    #df[str(treatment_variables[i])]= df[str(treatment_variables[i])].fillna(np.nanmedian(df[str(treatment_variables[i])]))
    print (smf.ols('md_earn_wne_p10 ~ ' + str(keep_variables[i]), data=df).fit().summary())

0
                            OLS Regression Results                            
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     67.50
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           1.24e-14
Time:                        22:32:27   Log-Likelihood:                -2700.6
No. Observations:                 246   AIC:                             5405.
Df Residuals:                     244   BIC:                             5412.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.848e+04   1978.611      9.339      0

                            OLS Regression Results                            
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.4737
Date:                Mon, 12 Mar 2018   Prob (F-statistic):              0.492
Time:                        22:32:27   Log-Likelihood:                -2516.2
No. Observations:                 233   AIC:                             5036.
Df Residuals:                     231   BIC:                             5043.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.163e+04    869.316     36.380      0.0

In [54]:
#correlations for '25k_or_above'
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['Binary_25k'].corr(df[str(keep_variables[i])])

PREDDEG
0.39636765679645297
CONTROL
-0.15163807316048447
LOCALE
-0.10347275931411307
DISTANCEONLY
0.09335200560186736
UGDS
0.22831962129001293
UGDS_WHITE
0.15834698601115635
UGDS_BLACK
-0.13374892564831256
UGDS_HISP
-0.17474408408610814
UGDS_ASIAN
0.25590365506299945
UGDS_AIAN
-0.09082501386437898
UGDS_NHPI
0.01235225856116062
UGDS_2MOR
0.038289320500239185
UGDS_NRA
0.20061515281816797
UGDS_UNKN
0.10901886982730172
PPTUG_EF
0.045929278837877265
PCTPELL
-0.24810142876650657
PCTFLOAN
0.15628626229871867
UG25abv
-0.14087163166738353
GRAD_DEBT_MDN_SUPP
0.48149686747241194
GRAD_DEBT_MDN10YR_SUPP
0.4814968674866805
NET_COST
0.21972206160949767


In [101]:
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['DISTANCEONLY'].corr(df[str(keep_variables[i])])

PREDDEG
0.10196840910372192
CONTROL
0.02513156188479263
LOCALE
0.02234081893229259
DISTANCEONLY
1.0
UGDS
0.3196618258623288
UGDS_WHITE
0.017568928015776926
UGDS_BLACK
0.007977324365745463
UGDS_HISP
-0.03483184967508984
UGDS_ASIAN
-0.02020105466184841
UGDS_AIAN
-0.0020880174753476624
UGDS_NHPI
0.0215383811980248
UGDS_2MOR
-0.016637355306038805
UGDS_NRA
-0.009601825782358308
UGDS_UNKN
0.03165792936168188
PPTUG_EF
0.1817823616945979
PCTPELL
-0.041558756999020854
PCTFLOAN
0.0049585742691977425
UG25abv
0.1193149642009548
GRAD_DEBT_MDN_SUPP
0.0653175403599605
GRAD_DEBT_MDN10YR_SUPP
0.06531754037193793
NET_COST
-0.05007035146339488


In [222]:
#regressions for 25k_or_above
for i in range(len(keep_variables)):
    print (i)
    #if i==3:
       # next
    
    print (smf.logit('Binary_25k ~ ' + str(keep_variables[i]), data=df).fit().summary())

0
Optimization terminated successfully.
         Current function value: 0.508251
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Binary_25k   No. Observations:                 4751
Model:                          Logit   Df Residuals:                     4749
Method:                           MLE   Df Model:                            1
Date:                Tue, 13 Mar 2018   Pseudo R-squ.:                  0.1741
Time:                        21:53:40   Log-Likelihood:                -2414.7
converged:                       True   LL-Null:                       -2923.7
                                        LLR p-value:                2.279e-223
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.3018      0.102    -32.512      0.000      -3.501      -3.103
PREDDEG        1.1987      

                           Logit Regression Results                           
Dep. Variable:             Binary_25k   No. Observations:                 4500
Model:                          Logit   Df Residuals:                     4498
Method:                           MLE   Df Model:                            1
Date:                Tue, 13 Mar 2018   Pseudo R-squ.:                 0.01829
Time:                        21:53:40   Log-Likelihood:                -2673.2
converged:                       True   LL-Null:                       -2723.0
                                        LLR p-value:                 1.854e-23
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0623      0.039    -27.476      0.000      -1.138      -0.987
UGDS_ASIAN     5.6666      0.620      9.133      0.000       4.450       6.883
9
Optimization terminated successfully.
         Cur

Optimization terminated successfully.
         Current function value: 0.538875
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Binary_25k   No. Observations:                 4273
Model:                          Logit   Df Residuals:                     4271
Method:                           MLE   Df Model:                            1
Date:                Tue, 13 Mar 2018   Pseudo R-squ.:                  0.1498
Time:                        21:53:40   Log-Likelihood:                -2302.6
converged:                       True   LL-Null:                       -2708.4
                                        LLR p-value:                1.707e-178
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -2.9207      0.097    -29.984      0.000      -3.112      

In [218]:
total = ''
for i in range(len(keep_variables)):
    if i== len(keep_variables)-1:
        total = total + str(keep_variables[i])
    elif str(keep_variables[i]) == 'GRAD_DEBT_MDN10YR_SUPP':
        next
    elif str(keep_variables[i]) == 'UGDS_UNKN':
        next
    else:
        total = total + str(keep_variables[i]) + ' + '

In [219]:
total

'PREDDEG + CONTROL + LOCALE + DISTANCEONLY + UGDS + UGDS_WHITE + UGDS_BLACK + UGDS_HISP + UGDS_ASIAN + UGDS_AIAN + UGDS_NHPI + UGDS_2MOR + UGDS_NRA + PPTUG_EF + PCTPELL + PCTFLOAN + UG25abv + GRAD_DEBT_MDN_SUPP + NET_COST'

In [77]:
smf.ols('RPY_3YR_RT_SUPP ~ ' + total, data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     20.87
Date:                Tue, 13 Mar 2018   Prob (F-statistic):           1.90e-38
Time:                        20:34:16   Log-Likelihood:                 180.03
No. Observations:                 231   AIC:                            -320.1
Df Residuals:                     211   BIC:                            -251.2
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.6285      0.093      6.725      0.000       0.444       0.813
PREDDEG                0.0580      0.018      3.173      0.002       0.022       0.094
CONTROL               -0.0361      0.017     -2.088      0.038      -0.070      -0.002
LOCALE                -0.0006      0.001     -0.549      0.584      -0.003       0.002
DISTANCEONLY           0.2283      0.163      1.397      0.164      -0.094       0.550
UGDS               -2.373e-06   2.33e-06     -1.019      0.309   -6.96e-06    2.22e-06
UGDS_WHITE             0.1982      0.064      3.089      0.002       0.072       0.325
UGDS_BLACK            -0.0756      0.071     -1.058      0.291      -0.217       0.065
UGDS_HISP              0.0025      0.072      0.035      0.972      -0.139       0.144
UGDS_ASIAN             1.1706      0.243      4.825      0.000       0.692       1.649
UGDS_AIAN             -1.1434      0.568     -2.013      0.045      -2.263      -0.024
UGDS_NHPI             -0.9855      1.448     -0.681      0.497      -3.839       1.869
UGDS_2MOR             -0.7754      0.337     -2.301      0.022      -1.440      -0.111
UGDS_NRA               0.3370      0.257      1.311      0.191      -0.170       0.843
PPTUG_EF              -0.0589      0.045     -1.313      0.191      -0.147       0.030
PCTPELL               -0.3451      0.076     -4.565      0.000      -0.494      -0.196
PCTFLOAN               0.2630      0.064      4.112      0.000       0.137       0.389
UG25abv               -0.1485      0.051     -2.913      0.004      -0.249      -0.048
GRAD_DEBT_MDN_SUPP -5.676e-06   1.74e-06     -3.253      0.001   -9.12e-06   -2.24e-06
NET_COST            3.372e-06   1.73e-06      1.952      0.052    -3.4e-08    6.78e-06
==============================================================================
Omnibus:                        3.523   Durbin-Watson:                   2.161
Prob(Omnibus):                  0.172   Jarque-Bera (JB):                4.117
Skew:                          -0.045   Prob(JB):                        0.128
Kurtosis:                       3.648   Cond. No.                     4.80e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.8e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [78]:
smf.ols('md_earn_wne_p10 ~ ' + total, data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     17.31
Date:                Tue, 13 Mar 2018   Prob (F-statistic):           2.01e-30
Time:                        20:37:41   Log-Likelihood:                -2050.3
No. Observations:                 200   AIC:                             4139.
Df Residuals:                     181   BIC:                             4201.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           2.638e+04   6070.375      4.346      0.000    1.44e+04    3.84e+04
PREDDEG              958.9986   1296.749      0.740      0.461   -1599.691    3517.689
CONTROL            -2026.0918   1251.807     -1.619      0.107   -4496.103     443.920
LOCALE               103.1043     71.543      1.441      0.151     -38.062     244.271
DISTANCEONLY         7.53e-10   6.07e-10      1.241      0.216   -4.44e-10    1.95e-09
UGDS                   0.1141      0.148      0.772      0.441      -0.178       0.406
UGDS_WHITE         -3252.5976   4224.624     -0.770      0.442   -1.16e+04    5083.250
UGDS_BLACK           516.5865   4615.859      0.112      0.911   -8591.227    9624.400
UGDS_HISP           -307.5507   4864.957     -0.063      0.950   -9906.874    9291.773
UGDS_ASIAN          8.443e+04   1.76e+04      4.785      0.000    4.96e+04    1.19e+05
UGDS_AIAN           2.343e+04   4.65e+04      0.504      0.615   -6.83e+04    1.15e+05
UGDS_NHPI          -1.265e+05   1.09e+05     -1.163      0.246   -3.41e+05    8.81e+04
UGDS_2MOR           -3.01e+04   2.21e+04     -1.360      0.176   -7.38e+04    1.36e+04
UGDS_NRA            2.048e+04   1.98e+04      1.036      0.302   -1.85e+04    5.95e+04
PPTUG_EF            1215.6493   3033.220      0.401      0.689   -4769.370    7200.668
PCTPELL            -2.539e+04   5176.018     -4.905      0.000   -3.56e+04   -1.52e+04
PCTFLOAN            1.247e+04   4285.602      2.909      0.004    4009.832    2.09e+04
UG25abv             5146.0346   3531.074      1.457      0.147   -1821.329    1.21e+04
GRAD_DEBT_MDN_SUPP     0.5043      0.123      4.088      0.000       0.261       0.748
NET_COST               0.0927      0.118      0.788      0.432      -0.139       0.325
==============================================================================
Omnibus:                       43.987   Durbin-Watson:                   1.881
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              128.214
Skew:                           0.896   Prob(JB):                     1.44e-28
Kurtosis:                       6.489   Cond. No.                     1.91e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.64e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [62]:
repay_pvalues = smf.ols('RPY_3YR_RT_SUPP ~ ' + total, data=df).fit().pvalues
smr.sandbox.stats.multicomp.multipletests(repay_pvalues, alpha=0.05, method='bonferroni')

(array([False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,
        False, False, False, False]),
 array([1.00000000e+00, 3.72974523e-02, 9.61414394e-01, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.72335291e-04, 7.36434578e-04, 9.13094129e-02, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00]),
 0.0023287975150316775,
 0.002272727272727273)

In [63]:
median_pvalues = smf.ols('md_earn_wne_p10 ~ ' + total, data=df).fit().pvalues
smr.sandbox.stats.multicomp.multipletests(median_pvalues, alpha=0.05, method='bonferroni')

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False]),
 array([5.78251325e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 5.83819863e-01, 5.84736189e-01,
        5.84459935e-01, 6.04946384e-01, 5.90467240e-01, 5.54752943e-01,
        5.77642812e-01, 5.89710101e-01, 5.84603484e-01, 1.00000000e+00,
        2.33741530e-05, 1.54139539e-01, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00]),
 0.0023287975150316775,
 0.002272727272727273)

In [220]:
smf.logit('Binary_25k ~ ' + total , data=df).fit().summary()

Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Binary_25k   No. Observations:                 3896
Model:                          Logit   Df Residuals:                     3876
Method:                           MLE   Df Model:                           19
Date:                Tue, 13 Mar 2018   Pseudo R-squ.:                  0.4475
Time:                        21:53:01   Log-Likelihood:                -1326.6
converged:                       True   LL-Null:                       -2400.9
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.5983      0.639     -2.501      0.012      -2.851      -0.346
PREDDEG                0.8133      0.114      7.143      0.000       0.590       1.036
CONTROL               -1.1036      0.107    -10.287      0.000      -1.314      -0.893
LOCALE                -0.0020      0.006     -0.334      0.738      -0.014       0.010
DISTANCEONLY           0.6921      0.752      0.920      0.357      -0.782       2.166
UGDS                2.951e-05   1.03e-05      2.860      0.004    9.29e-06    4.97e-05
UGDS_WHITE            -0.0895      0.480     -0.186      0.852      -1.030       0.851
UGDS_BLACK            -0.1907      0.521     -0.366      0.714      -1.212       0.831
UGDS_HISP              0.5148      0.591      0.871      0.384      -0.643       1.673
UGDS_ASIAN             8.3411      1.271      6.561      0.000       5.849      10.833
UGDS_AIAN             -9.9573      3.849     -2.587      0.010     -17.502      -2.413
UGDS_NHPI              3.2040      2.909      1.101      0.271      -2.497       8.905
UGDS_2MOR             -4.1091      2.101     -1.956      0.050      -8.226       0.008
UGDS_NRA              -3.7994      1.596     -2.381      0.017      -6.927      -0.672
PPTUG_EF              -1.9268      0.277     -6.954      0.000      -2.470      -1.384
PCTPELL               -9.9747      0.567    -17.591      0.000     -11.086      -8.863
PCTFLOAN               5.9576      0.453     13.152      0.000       5.070       6.845
UG25abv                4.3175      0.343     12.580      0.000       3.645       4.990
GRAD_DEBT_MDN_SUPP  2.313e-05    1.1e-05      2.103      0.035    1.57e-06    4.47e-05
NET_COST            4.038e-05   1.09e-05      3.713      0.000    1.91e-05    6.17e-05
======================================================================================
"""

In [191]:
binary_pvalues = smf.logit('Binary_25k ~ ' + total , data=df).fit().pvalues
smr.sandbox.stats.multicomp.multipletests(binary_pvalues, alpha=0.05, method='bonferroni')

Optimization terminated successfully.
         Current function value: 0.275374
         Iterations 9


(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True,  True, False, False,
        False]),
 array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.81094482, 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.00125646,
        0.03176511, 1.        , 0.08688057, 0.47710175]),
 0.0026960063028712566,
 0.002631578947368421)

In [192]:
np.mean(df['Binary_25k'])

0.2777777777777778